## 30 Transformação de projeção ortogonal

Neste exemplo, a matriz de projeção ortogonal é utilizada, e seus parâmetros são alterados para renderizar uma mesma cena de diferentes formas. A função utilizada para cria essa matriz é a função glm.ortho(left, right, bottom, top, near, far). Também  é utilizada uma matriz de visão e seu parâmetros não são alterados ao longo do exemplo.

In [6]:
import glm
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v2 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v2 import ModelRenderer
from cg.models.SphereMesh_v1 import SphereMesh
from cg.models.SquareMesh_v1 import SquareMesh

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        # posiciona a câmera na posição z = 1 olhando para o eixo z engativo
        self.cameraEye   = glm.vec3(0.0, 0.0, 1.0) # posição da câmera
        self.cameraCenter = glm.vec3(0.0, 0.0, -1.0) # ponto de visão (para onde a câmera está olhando) 
        self.cameraUp    = glm.vec3(0.0, 1.0,  0.0) # vetor 'pra cima'
        self.viewMatrix = glm.lookAt(self.cameraEye, self.cameraCenter, self.cameraUp)
        
        # cria uma malha esférica
        sphere_mesh = SphereMesh(0.2, 20, 20)
        
        # cria uma malha quadrada
        square_mesh = SquareMesh(1.96, 1.96, 3, 3)
        
        # cria o objeto responsável por carregar os dados para a GPU e renderizá-los
        self.sphereRenderer = ModelRenderer(sphere_mesh.getVertexPositions(), vertex_indices=sphere_mesh.getVertexIndices())
        self.squareRenderer = ModelRenderer(square_mesh.getVertexPositions(), vertex_indices=square_mesh.getVertexIndices())
        
        # cria um shader program simples
        self.shaderProgram = SimpleShaderProgram()
        
        # ativa o shader programa para configurar uma cor única para todos os vértices
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.release()

        # recupera o endereço da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        
        # configura os dados do modelo para serem os dados de entrada do shader program
        self.sphereRenderer.setVertexPositionLoc(position_loc)
        self.squareRenderer.setVertexPositionLoc(position_loc)
        
        # habilita teste de profundidade e culling
        gl.glEnable(gl.GL_DEPTH_TEST);
        gl.glEnable(gl.GL_CULL_FACE)
        
    def paintGL(self):
        
        # configura a cor de background
        gl.glClearColor(1, 1, 1, 1)
        
        # limpa o background com a cor especificada e o buffer de profundidade
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # atualiza o viewport para a cena ser renderizado mais a esquerda
        gl.glViewport(0, 0, int(self.width / 5), self.height)
        
        # cria uma projeção ortogonal do tamanho do volume de renderização
        self.orthoMatrix = glm.ortho(-1, 1, -1, 1, 0, 2)
        
        # renderiza a esfera
        self.renderScene()
        
        # atualiza o viewport para a cena ser renderizado do lado esquerdo
        gl.glViewport(int(self.width / 5), 0, int(self.width / 5), self.height)
        
        # dobra o tamanho do cubo de projeção nos eixo x e y (usa a primeira cena da esquerda como base)
        # deixando as esferas menores e mostrando a esfera verde
        self.orthoMatrix = glm.ortho(-2, 2, -2, 2, 0, 2)
        
        # renderiza a cena
        self.renderScene()
        
        # atualiza o viewport para a cena ser renderizado no centro
        gl.glViewport(int(2 * self.width / 5), 0, int(self.width / 5), self.height)
        
        # aumenta a distância do plano 'longe' (far plane)(usa a primeira cena da esquerda como base)
        # com isso, a esfera laranja é renderizada
        self.orthoMatrix = glm.ortho(-1, 1, -1, 1, 0, 3)
        
        # renderiza a cena
        self.renderScene()
        
        # atualiza o viewport para a cena ser renderizado no lado direito
        gl.glViewport(int(3 * self.width / 5), 0, int(self.width / 5), self.height)
        
        # diminui a distância do plano 'perto' (near plane)(usa a cena central como base)
        # com isso, a esfera preta é renderizada
        # Com a projeção ortogonal, é possível utilizar um 'near plane' com valor negativo
        # para mostra objetos que estão atrás da câmera
        self.orthoMatrix = glm.ortho(-1, 1, -1, 1, -1, 3)
        
        # renderiza a cena
        self.renderScene()
        
        # atualiza o viewport para a cena ser renderizado mais a direita
        gl.glViewport(int(4 * self.width / 5), 0, int(self.width / 5), self.height)
        
        # dobra o tamanho do cubo de projeção nos eixo x e y (usa a quarta cena como base)
        # com isso, todos os objetos da cena são renderizados
        self.orthoMatrix = glm.ortho(-2, 2, -2, 2, -1, 3)
        
        # renderiza a cena
        self.renderScene()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
    
    def renderScene(self):
        
        self.renderSphere(glm.vec3(0.0, 0.0, -0.5), np.array([0.0, 0.0, 0.5, 1.0])) # esfera azul
        self.renderSphere(glm.vec3(-1.2, -0.8, 0.2), np.array([0.0, 0.5, 0.0, 1.0]))# esfera verde
        self.renderSphere(glm.vec3(-0.5, -0.2, 0.6), np.array([0.5, 0.0, 0.0, 1.0]))# esfera vermelha
        self.renderSphere(glm.vec3(0.3, -0.2, -0.3), np.array([0.8, 0.8, 0.0, 1.0]))# esfera amarela
        self.renderSphere(glm.vec3(-0.6, 0.7, -0.5), np.array([0.0, 0.5, 0.5, 1.0]))# esfera ciano
        self.renderSphere(glm.vec3(0.5, 0.2, 0.3), np.array([0.5, 0.0, 0.5, 1.0]))  # esfera roxa
        self.renderSphere(glm.vec3(0.1, 0.8, -1.2), np.array([0.8, 0.4, 0.0, 1.0])) # esfera laranja
        self.renderSphere(glm.vec3(0.0, -0.8, 1.2), np.array([0.0, 0.0, 0.0, 1.0]))# esfera preta
        
        self.renderSquare()
        
    def renderSphere(self, transl, color):
        
        # calcula a matriz de translação da esfera e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), transl)
        self.shaderProgram.setUniformMVPMatrix(self.orthoMatrix * self.viewMatrix * model_matrix)
        
        # mudar a cor no shader e renderiza a esfera
        self.shaderProgram.setUniformColor(color)
        self.sphereRenderer.render()
    
    def renderSquare(self):
        
        # calcula a matriz de translação do quadrado para ele ficar no fundo da cena
        # Atualiza a matriz do shader apenas com a matriz de modelo
        model_matrix = glm.translate(glm.mat4(), glm.vec3(0.0, 0.0, 0.999))
        self.shaderProgram.setUniformMVPMatrix(model_matrix)
        
        # mudar a cor no shader e renderiza o quadrado
        self.shaderProgram.setUniformColor(np.array([0.9, 0.9, 0.9, 1.0]))
        self.squareRenderer.render()
        
    def resizeGL(self, width, height):

        # armazena a largura e a altura da janela
        self.width = width
        self.height = height

def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(1500, 300)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
! jupyter nbconvert --to python 30_Transformacao_projecao_ortogonal.ipynb
%run -i 30_Transformacao_projecao_ortogonal.py